In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
import requests
from bs4 import BeautifulSoup
import re

In [3]:
df=pd.read_excel("./cik_list.xlsx")

In [4]:
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [5]:
df.shape

(152, 6)

In [6]:
#adding "https://www.sec.gov/Archives/" in SECfNAME to fetch reports from link
df.SECFNAME='https://www.sec.gov/Archives/'+df.SECFNAME

In [7]:
df.SECFNAME.head()

0    https://www.sec.gov/Archives/edgar/data/3662/0...
1    https://www.sec.gov/Archives/edgar/data/3662/0...
2    https://www.sec.gov/Archives/edgar/data/3662/0...
3    https://www.sec.gov/Archives/edgar/data/3662/0...
4    https://www.sec.gov/Archives/edgar/data/3662/0...
Name: SECFNAME, dtype: object

In [8]:
#stop words
with open('StopWords_Generic.txt','r') as s:
    stop=s.read()
stop=stop.split('\n')

In [9]:
len(stop)
# stop

121

In [10]:
dict = pd.read_excel('LoughranMcDonald_MasterDictionary_2018.xlsx')

In [11]:
dict.head()

,Word,Sequence Number,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Constraining,Superfluous,Interesting,Modal,Irr_Verb,Harvard_IV,Syllables,Source
0,AARDVARK,1,277,1.480368e-08,1.239377e-08,3.564730e-06,84,0,0,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.603287e-10,9.725110e-12,9.863549e-09,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,8,4.275431e-10,1.386497e-10,6.225591e-08,7,0,0,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,12,6.413147e-10,3.159061e-10,9.383557e-08,12,0,0,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,7250,3.874610e-07,3.681624e-07,3.366553e-05,914,0,0,0,0,0,0,0,0,0,0,3,12of12inf


In [12]:
# total positive and negative words in master dictionary
pos=[x for x in dict[dict['Positive']!=0]['Word']]
neg=[x for x in dict[dict['Negative']!=0]['Word']]
len(pos)

354

In [13]:
len(neg)

2355

In [14]:
uncertainity=pd.read_excel('uncertainty_dictionary.xlsx')

In [15]:
uncertainity.head()

,Word
0,ABEYANCE
1,ABEYANCES
2,ALMOST
3,ALTERATION
4,ALTERATIONS


In [16]:
uncertainity_words = list(uncertainity['Word'])

In [17]:
constraining=pd.read_excel('constraining_dictionary.xlsx')
constraining.head()

,Word
0,ABIDE
1,ABIDING
2,BOUND
3,BOUNDED
4,COMMIT


In [18]:
constraining_words = list(constraining['Word'])

In [19]:
links=[x for x in df['SECFNAME']]
# links

In [20]:
# fetching reports from links using scrapper beautiful soup
reports=[]
for i in links:
    r=requests.get(i)
    data=r.text
    a=BeautifulSoup(data, "html.parser")
    reports.append(a.get_text())

In [21]:
len(reports)

152

In [22]:
# output variables
col=['positive_score','negative_score','polarity_score','average_sentence_length','percentage_of_complex_words',
     'fog_index','complex_word_count','word_count','uncertainity_score','constraining_score','positive_word_proportion',
      'negative_word_proportion','uncertainity_word_proportion','constraining_word_proportion','constraining_words_whole_report']

#inserting output variable columns with columns of “cik_list.xlsx”
for c in col:
    df[c]=0.0;

In [23]:
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average_sentence_length,...,fog_index,complex_word_count,word_count,uncertainity_score,constraining_score,positive_word_proportion,negative_word_proportion,uncertainity_word_proportion,constraining_word_proportion,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
#spliting reports into list of words
def word_tok(text):
    text=re.sub(r"[^A-Za-z]"," ",text.upper())
    token=word_tokenize(text)
    return token

#spliting reports into list of sentences
def sent_tok(text):
    sent=sent_tokenize(text)
    num_sent=len(sent)
    return num_sent

# removing stopwords
def stop_words(words, stop_words):
    return [x for x in words if x not in stop_words]

# counting no. of words present in set of words that are also present in orignal dictionary
def countfunc(orig, words):
    count = 0
    for x in words:
        if(x in orig):
            count+=1
    return count

#calculating polarity score
def polarity(pos_score, neg_score):
    pol=(pos_score - neg_score)/((pos_score + neg_score)+ 0.000001)
    return pol

# calculaing subjectivity score
def subjectivity(pos_score, neg_score, num_words):
    subj=(pos_score+neg_score)/(num_words+ 0.000001)
    return subj

# Sentiment score categorization    
def sentiment(pol):
    if(pol<-0.5):
        return "most negative"
    elif(pol>=-0.5 and pol<0):
        return "negative"
    elif(pol==0):
        return "neutral"
    elif(pol>0 and pol<0.5):
        return "positive"
    else:
        return "very positive"

# average sentence length
# def avg_sent_len(num_words,num_sent):
#     return (num_words/num_sent)

#counting syllables
from nltk.corpus import cmudict
d = cmudict.dict()

def nsyl(words):
    num=0
    for word in words:
        word = word.lower()
        count = 0
        vowels = "aeiouy"
        if word[0] in vowels:
            count += 1
        for index in range(1, len(word)):
            if word[index] in vowels and word[index - 1] not in vowels:
                count += 1
        if word.endswith("e"):
            count -= 1
        if count == 0:
            count += 1
        if count>2:
            num+=1
    return num
    
#fog index calculation
def fog_ind(avg_sent_len, perc_comp_words):
    return (0.4*(avg_sent_len + perc_comp_words))

# proportion calculation
def prop_cal(score,words):
    return (score/words)

In [ ]:
#calculating all output variables for each report 
for i in range(len(reports)):
    token=word_tok(reports[i])
    num_sent=sent_tok(reports[i])
    words=stop_words(token,stop)
    num_words=len(words)
    pos_score=countfunc(pos,words)
    neg_score=countfunc(neg,words)
    pol=polarity(pos_score,neg_score)
    subj=subjectivity(pos_score,neg_score,num_words)
    avg_sent_len=(num_words/num_sent)
    num_comp=nsyl(words)
    perc_comp_words=(num_comp/num_sent)
    fog_ind=0.4*(avg_sent_len + perc_comp_words)
    uncer_score=countfunc(uncertainity_words,words)
    const_score=countfunc(constraining_words,words)
    pos_word_prop=(pos_score/num_words)
    neg_word_prop=(neg_score/num_words)
    uncer_word_prop=(uncer_score/num_words)
    const_word_prop=(const_score/num_words)
    
    df.at[i,'positive_score']=pos_score
    df.at[i,'negative_score']=neg_score
    df.at[i,'polarity_score']=pol
    df.at[i,'average_sentence_length']=avg_sent_len
    df.at[i,'percentage_of_complex_words']=perc_comp_words      
    df.at[i,'fog_index']=fog_ind      
    df.at[i,'complex_word_count']=num_comp
    df.at[i,'word_count']=num_words
    df.at[i,'uncertainity_score']=uncer_score
    df.at[i,'constraining_score']=const_score
    df.at[i,'positive_word_proportion']=pos_word_prop
    df.at[i,'negative_word_proportion']=neg_word_prop
    df.at[i,'uncertainity_word_proportion']=uncer_word_prop
    df.at[i,'constraining_word_proportion']=const_word_prop
    df.at[i,'constraining_words_whole_report']=const_score
          
          
          

In [ ]:
df.head()

In [ ]:
df.to_excel('output.xlsx')